In [1]:
import requests
import pandas as pd
import json
import tensorflow as tf
import base64

In [2]:
tweet_data = pd.read_csv('data/data.csv')

In [3]:
tweet_test = tweet_data.tweet[10]
tweet_test

"spring break in plain city... it's snowing "

In [4]:
tweet_label = tweet_data.label[10]
tweet_label

0

### Make prediction request

In [5]:
json_data = json.dumps(
    {
        "data": tweet_test
    }
)

In [6]:
endpoint = "http://103.190.214.113:5000/predict"
headers = {"content-type": "application/json"}
response = requests.post(endpoint, data=json_data, headers=headers)
print('Tweet:', response.json()['tweet'])
print('Result:', response.json()['result'])

Tweet: spring break in plain city... it's snowing 
Result: Negative


In [7]:
model_server_url = "http://103.190.214.113:8501/v1/models/cc-model:predict"

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def serialize_text(text):
    example = tf.train.Example(features=tf.train.Features(feature={
        'tweet': _bytes_feature(text)
        }))
    serialized_example = example.SerializeToString()
    return serialized_example

def predict(text):
    example = serialize_text(text)
    json_data = {
      "signature_name":"serving_default",
      "instances":[
        {
          "examples":{"b64": base64.b64encode(example).decode('utf-8')}
        }
      ]
    }
    response = requests.post(model_server_url, json=json_data)
    predictions = json.loads(response.text)['predictions']
    return predictions

In [8]:
instances = tweet_test
text = str.encode(instances)
print(predict(text))

[[0.968495846]]
